In [92]:
import pandas as pd

In [93]:
df = pd.read_csv("jobs.csv")
df

,Company,Title,Location,Type,level,Salary,Requirment,Facilities,Country,cluster,x0,x1,classe
0,SGS,clinical data analyst,"Richardson, TX, United States",Full Time,Junior,48,computer science data quality genetics mathema...,NaN,United States,Machine Learning and Computer Vision,0.093032,0.071434,Machine Learning and Computer Vision
1,Bosch Group,application developer data analyst,"Nonantola, Italy",Full Time,Junior,48,engineering industrial oracle power bi r r,NaN,Italy,Machine Learning and Computer Vision,0.217672,0.116621,Machine Learning and Computer Vision
2,Publicis Groupe,data engineer full time public sector,"Arlington, VA, United States",Full Time,Intermediate,108,aws azure computer science consulting dataflow...,flex hours flex vacation parental leave unlimi...,United States,Data Visualization and Business Analytics,-0.179149,-0.198149,Data Visualization and Business Analytics
3,ServiceNow,data scientist,"Kirkland, Washington, United States",Full Time,Senior,184,computer science deep learning industrial mach...,k matching career development competitive pay ...,United States,Cloud Computing and Data Science,0.405995,-0.211615,Cloud Computing and Data Science
4,Natixis in Portugal,vendor management data quality lead,"Porto, Portugal",Full Time,Junior,39,banking data quality excel security,gear,Portugal,Machine Learning and Computer Vision,0.059897,0.231600,Machine Learning and Computer Vision
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2310,CCRi,application integration engineer computer visi...,"Chantilly, Virginia, United States",Full Time,Intermediate,113,agile angular apis architecture aws azure,k matching career development flex hours flex ...,United States,Data Visualization and Business Analytics,-0.244297,-0.304274,Data Visualization and Business Analytics
2311,Publicis Groupe,associate data science,"New York City, United States",Full Time,Intermediate,106,bayesian classification clustering data analys...,career development health care,United States,Cloud Computing and Data Science,0.243486,0.008272,Cloud Computing and Data Science
2312,DoorDash,software engineer machine learning intelligence,"Sunnyvale, CA; San Francisco, CA; New York",Full Time,Senior,176,computer science data analysis engineering exc...,k matching career development equity insurance...,United States,Machine Learning and Computer Vision,0.134405,0.066488,Machine Learning and Computer Vision
2313,Western Digital,data scientist college graduate,"Biñan, Philippines",Full Time,Junior,39,apis clustering computer science data visualiz...,career development,Philippines,Cloud Computing and Data Science,0.136699,-0.162053,Cloud Computing and Data Science


In [94]:
df.dropna(inplace=True)

In [95]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1959 entries, 0 to 1958
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Company     1959 non-null   object 
 1   Title       1959 non-null   object 
 2   Location    1959 non-null   object 
 3   Type        1959 non-null   object 
 4   level       1959 non-null   object 
 5   Salary      1959 non-null   int64  
 6   Requirment  1959 non-null   object 
 7   Facilities  1959 non-null   object 
 8   Country     1959 non-null   object 
 9   cluster     1959 non-null   object 
 10  x0          1959 non-null   float64
 11  x1          1959 non-null   float64
 12  classe      1959 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 199.1+ KB


In [97]:
len(df['Requirment'].unique())

1741

In [98]:
import pandas as pd
import pyodbc

# Replace the connection string with your actual SQL Server connection string
conn_str = 'DRIVER={SQL Server};SERVER=LAPTOP-REU1GLF5\SQLEXPRESS;DATABASE=Jobs'

# Establish a connection to the database
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [99]:
cursor.execute("IF OBJECT_ID('company_jobs', 'U') IS NOT NULL DROP TABLE company_jobs")
cursor.execute("IF OBJECT_ID('Description', 'U') IS NOT NULL DROP TABLE Description")
cursor.execute("IF OBJECT_ID('Requirement', 'U') IS NOT NULL DROP TABLE Requirement")
cursor.execute("IF OBJECT_ID('Jobs', 'U') IS NOT NULL DROP TABLE Jobs")
cursor.execute("IF OBJECT_ID('company_location', 'U') IS NOT NULL DROP TABLE company_location")
cursor.execute("IF OBJECT_ID('Location', 'U') IS NOT NULL DROP TABLE Location")
cursor.execute("IF OBJECT_ID('Company', 'U') IS NOT NULL DROP TABLE Company")

conn.commit() 

In [100]:
# Create the Languages table if it doesn't exist
cursor.execute('''
CREATE TABLE Company (
    company_id INT PRIMARY KEY,
    company_name VARCHAR(100)
)
''')
conn.commit()

In [101]:
unique_company = df['Company'].unique()
for idx, company in enumerate(unique_company):
    cursor.execute("INSERT INTO Company (company_id, company_name) VALUES (?, ?)", idx + 1, company)
    conn.commit()


In [102]:
# Create the Languages table if it doesn't exist
cursor.execute('''
CREATE TABLE Location (
    country_id INT PRIMARY KEY,
    country_name VARCHAR(50)
)
''')
conn.commit()

In [103]:
unique_country = df['Country'].unique()
for idx, country in enumerate(unique_country):
    cursor.execute("INSERT INTO Location (country_id, country_name) VALUES (?, ?)", idx + 1, country)
    conn.commit()


In [104]:
len(unique_country)

78

In [105]:
# Create the Repository_Languages table with foreign keys to Repositories and Languages tables
cursor.execute('''
CREATE TABLE company_location (
    company_id INT,
    country_id INT,
    PRIMARY KEY (company_id, country_id),
    FOREIGN KEY (company_id) REFERENCES Company(company_id),
    FOREIGN KEY (country_id) REFERENCES Location(country_id)
)
''')
conn.commit()

In [106]:
df1 = df[["Company", "Country"]]

In [107]:
unique_country = pd.DataFrame(unique_country)
unique_company = pd.DataFrame(unique_company)

In [108]:
unique_country.reset_index(inplace=True)
unique_company.reset_index(inplace=True)

In [109]:
unique_country.rename(columns={"index":"country_id", 0:"Country"}, inplace=True)
unique_company.rename(columns={"index":"company_id", 0:"Company"}, inplace=True)

In [110]:
unique_country["country_id"] = unique_country["country_id"] + 1
unique_company["company_id"] = unique_company["company_id"] + 1

In [111]:
unique_country

,country_id,Country
0,1,United States
1,2,Portugal
2,3,Switzerland
3,4,India
4,5,Germany
...,...,...
73,74,Taiwan
74,75,Albania
75,76,Bangladesh
76,77,Columbia


In [112]:
unique_company

,company_id,Company
0,1,Publicis Groupe
1,2,ServiceNow
2,3,Natixis in Portugal
3,4,Palo Alto Networks
4,5,CERN
...,...,...
795,796,23andMe
796,797,WITHIN
797,798,DrDoctor
798,799,Thaloz


In [113]:
result_df = pd.merge(df1, unique_country, on='Country', how='left')

In [114]:
result_df = pd.merge(result_df, unique_company, on='Company', how='left')

In [115]:
result_df

,Company,Country,country_id,company_id
0,Publicis Groupe,United States,1,1
1,ServiceNow,United States,1,2
2,Natixis in Portugal,Portugal,2,3
3,Palo Alto Networks,United States,1,4
4,CERN,Switzerland,3,5
...,...,...,...,...
1954,CCRi,United States,1,215
1955,Publicis Groupe,United States,1,1
1956,DoorDash,United States,1,433
1957,Western Digital,Philippines,32,15


In [116]:
comp_loc_data = []
for row in result_df.itertuples(index=False):
    country_id = row.country_id
    company_id = row.company_id
    comp_loc_data.append((country_id, company_id))

In [117]:
print(comp_loc_data)

[(1, 1), (1, 2), (2, 3), (1, 4), (3, 5), (4, 6), (2, 7), (5, 7), (1, 2), (1, 8), (6, 9), (4, 10), (1, 7), (1, 11), (1, 1), (7, 12), (8, 13), (9, 14), (10, 15), (11, 16), (1, 17), (4, 10), (5, 18), (12, 19), (1, 20), (1, 21), (13, 22), (4, 23), (14, 24), (1, 25), (1, 26), (14, 27), (8, 7), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (15, 33), (1, 34), (1, 35), (14, 7), (1, 36), (1, 37), (16, 38), (17, 39), (6, 40), (1, 41), (1, 42), (5, 43), (16, 44), (18, 45), (19, 46), (1, 1), (20, 47), (1, 31), (1, 36), (19, 46), (1, 17), (1, 10), (13, 48), (1, 49), (13, 48), (16, 50), (4, 51), (1, 29), (1, 52), (1, 53), (1, 35), (5, 54), (21, 55), (18, 56), (1, 17), (6, 57), (1, 1), (14, 58), (1, 59), (16, 44), (1, 60), (1, 61), (22, 62), (1, 12), (1, 2), (1, 63), (1, 64), (12, 19), (1, 65), (1, 21), (4, 37), (1, 66), (1, 67), (14, 68), (6, 69), (1, 1), (1, 70), (2, 7), (13, 48), (1, 71), (11, 72), (4, 6), (16, 73), (4, 74), (14, 70), (14, 75), (5, 16), (1, 76), (16, 12), (23, 77), (1, 78), (16, 79

In [118]:
print(len(comp_loc_data))

1959


In [119]:
comp_loc = list(set(comp_loc_data))

In [120]:
print(len(comp_loc))

1023


In [121]:
  for data in comp_loc:
    cursor.execute("INSERT INTO company_location (country_id, company_id) VALUES (?, ?)", data[0], data[1])
    conn.commit()


In [122]:
df.columns

Index(['Company', 'Title', 'Location', 'Type', 'level', 'Salary', 'Requirment',
       'Facilities', 'Country', 'cluster', 'x0', 'x1', 'classe'],
      dtype='object')

In [123]:
df2 = df[['Title', 'Type', 'Salary']]

In [124]:
df2

,Title,Type,Salary
0,data engineer full time public sector,Full Time,108
1,data scientist,Full Time,184
2,vendor management data quality lead,Full Time,39
3,principal cloud data engineer prisma access,Full Time,140
4,data scientist grap,Full Time,40
...,...,...,...
1954,application integration engineer computer visi...,Full Time,113
1955,associate data science,Full Time,106
1956,software engineer machine learning intelligence,Full Time,176
1957,data scientist college graduate,Full Time,39


In [125]:
unique_jobs = df2['Title'].unique()

In [126]:
len(unique_jobs)

1145

In [127]:
unique_jobs = pd.DataFrame(unique_jobs)
unique_jobs.reset_index(inplace=True)
unique_jobs.rename(columns={"index":"jobs_id", 0:"Title"}, inplace=True)
unique_jobs["jobs_id"] = unique_jobs["jobs_id"] + 1
unique_jobs

,jobs_id,Title
0,1,data engineer full time public sector
1,2,data scientist
2,3,vendor management data quality lead
3,4,principal cloud data engineer prisma access
4,5,data scientist grap
...,...,...
1140,1141,people data specialist
1141,1142,machine learning engineer trip planning
1142,1143,application integration engineer computer visi...
1143,1144,software engineer machine learning intelligence


In [128]:
# Create the table if it doesn't exist
cursor.execute('''
CREATE TABLE Jobs (
    job_id INT PRIMARY KEY,
    Title VARCHAR(100)
)
''')
conn.commit()


In [129]:
# Insert data into the table
for row in unique_jobs.itertuples(index=False):
    cursor.execute("INSERT INTO Jobs (job_id, Title) VALUES (?, ?)", 
                   row.jobs_id, row.Title)
    conn.commit()

In [130]:
jobs_company = pd.merge(unique_jobs, df, on='Title', how='left')

In [131]:
jobs_company

,jobs_id,Title,Company,Location,Type,level,Salary,Requirment,Facilities,Country,cluster,x0,x1,classe
0,1,data engineer full time public sector,Publicis Groupe,"Arlington, VA, United States",Full Time,Intermediate,108,aws azure computer science consulting dataflow...,flex hours flex vacation parental leave unlimi...,United States,Data Visualization and Business Analytics,-0.179149,-0.198149,Data Visualization and Business Analytics
1,2,data scientist,ServiceNow,"Kirkland, Washington, United States",Full Time,Senior,184,computer science deep learning industrial mach...,k matching career development competitive pay ...,United States,Cloud Computing and Data Science,0.405995,-0.211615,Cloud Computing and Data Science
2,2,data scientist,ServiceNow,"Kirkland, Washington, United States",Full Time,Senior,159,computer science deep learning industrial mach...,k matching career development competitive pay ...,United States,Cloud Computing and Data Science,0.405995,-0.211615,Cloud Computing and Data Science
3,2,data scientist,Sunnova Energy International,United States,Full Time,Senior,129,agile apis architecture aws big data classific...,career development,United States,Data Visualization and Business Analytics,-0.325900,-0.263721,Data Visualization and Business Analytics
4,2,data scientist,Monzo,"Cardiff, London or Remote (UK)",Full Time,Senior,129,b testing banking data mining finance looker p...,career development conferences flex hours,United Kingdom,Machine Learning and Computer Vision,0.150298,0.202743,Machine Learning and Computer Vision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,1141,people data specialist,Marley Spoon,"Lisbon, Lisbon, Portugal",Full Time,Intermediate,70,data analysis excel finance,career development equity flex hours flex vaca...,Portugal,Machine Learning and Computer Vision,0.065655,0.374789,Machine Learning and Computer Vision
1955,1142,machine learning engineer trip planning,TripAdvisor,"Needham, Massachusetts, MA",Full Time,Senior,170,big data computer science computer vision engi...,career development conferences flex hours flex...,United States,Machine Learning and Computer Vision,0.062663,-0.138996,Machine Learning and Computer Vision
1956,1143,application integration engineer computer visi...,CCRi,"Chantilly, Virginia, United States",Full Time,Intermediate,113,agile angular apis architecture aws azure,k matching career development flex hours flex ...,United States,Data Visualization and Business Analytics,-0.244297,-0.304274,Data Visualization and Business Analytics
1957,1144,software engineer machine learning intelligence,DoorDash,"Sunnyvale, CA; San Francisco, CA; New York",Full Time,Senior,176,computer science data analysis engineering exc...,k matching career development equity insurance...,United States,Machine Learning and Computer Vision,0.134405,0.066488,Machine Learning and Computer Vision


In [132]:
jobs_company = pd.merge(jobs_company, unique_company, on='Company', how='left')

In [133]:
jobs_company.columns

Index(['jobs_id', 'Title', 'Company', 'Location', 'Type', 'level', 'Salary',
       'Requirment', 'Facilities', 'Country', 'cluster', 'x0', 'x1', 'classe',
       'company_id'],
      dtype='object')

In [134]:
jobs_company = jobs_company[["Title", "jobs_id", "Company", "company_id"]]

In [135]:
jobs_company

,Title,jobs_id,Company,company_id
0,data engineer full time public sector,1,Publicis Groupe,1
1,data scientist,2,ServiceNow,2
2,data scientist,2,ServiceNow,2
3,data scientist,2,Sunnova Energy International,21
4,data scientist,2,Monzo,24
...,...,...,...,...
1954,people data specialist,1141,Marley Spoon,460
1955,machine learning engineer trip planning,1142,TripAdvisor,85
1956,application integration engineer computer visi...,1143,CCRi,215
1957,software engineer machine learning intelligence,1144,DoorDash,433


In [136]:
# Create the Repository_Languages table with foreign keys to Repositories and Languages tables
cursor.execute('''
CREATE TABLE company_jobs (
    company_id INT,
    job_id INT,
    PRIMARY KEY (company_id, job_id),
    FOREIGN KEY (company_id) REFERENCES Company(company_id),
    FOREIGN KEY (job_id) REFERENCES Jobs(job_id)
)
''')
conn.commit()

In [137]:
comp_loc_data = []
for row in jobs_company.itertuples(index=False):
    job_id = row.jobs_id
    company_id = row.company_id
    comp_loc_data.append((job_id, company_id))

In [138]:
comp_loc = list(set(comp_loc_data))

In [139]:
len(comp_loc)

1742

In [140]:
for data in comp_loc:
    cursor.execute("INSERT INTO company_jobs (job_id, company_id) VALUES (?, ?)", data[0], data[1])
    conn.commit()

In [141]:
len(unique_req)

1741

In [142]:
unique_req = df['Requirment'].unique()
unique_req = pd.DataFrame(unique_req)
unique_req.reset_index(inplace=True)
unique_req.rename(columns={"index":"req_id", 0:"Requirment"}, inplace=True)
unique_req["req_id"] = unique_req["req_id"] + 1
unique_req

# Create the table if it doesn't exist
cursor.execute('''
CREATE TABLE Requirement (
    req_id INT PRIMARY KEY,
    requirement VARCHAR(500)
)
''')
conn.commit()

# Insert data into the table
for row in unique_req.itertuples(index=False):
    cursor.execute("INSERT INTO Requirement (req_id, requirement) VALUES (?, ?)", 
                   row.req_id, row.Requirment)
    conn.commit()



In [143]:
df.columns

Index(['Company', 'Title', 'Location', 'Type', 'level', 'Salary', 'Requirment',
       'Facilities', 'Country', 'cluster', 'x0', 'x1', 'classe'],
      dtype='object')

In [144]:
df2 = df[['Title', 'Type', 'level', 'Salary', 'classe', 'Requirment']]

In [145]:
result_df2 = pd.merge(df2, unique_jobs, on='Title', how='left')

In [146]:
result_df2 = pd.merge(result_df2, unique_req, on='Requirment', how='left')

In [147]:
result_df2

,Title,Type,level,Salary,classe,Requirment,jobs_id,req_id
0,data engineer full time public sector,Full Time,Intermediate,108,Data Visualization and Business Analytics,aws azure computer science consulting dataflow...,1,1
1,data scientist,Full Time,Senior,184,Cloud Computing and Data Science,computer science deep learning industrial mach...,2,2
2,vendor management data quality lead,Full Time,Junior,39,Machine Learning and Computer Vision,banking data quality excel security,3,3
3,principal cloud data engineer prisma access,Full Time,Senior,140,Data Visualization and Business Analytics,agile apis aws azure big data computer science,4,4
4,data scientist grap,Full Time,Junior,40,Cloud Computing and Data Science,computer science data analysis deep learning k...,5,5
...,...,...,...,...,...,...,...,...
1954,application integration engineer computer visi...,Full Time,Intermediate,113,Data Visualization and Business Analytics,agile angular apis architecture aws azure,1143,323
1955,associate data science,Full Time,Intermediate,106,Cloud Computing and Data Science,bayesian classification clustering data analys...,501,1738
1956,software engineer machine learning intelligence,Full Time,Senior,176,Machine Learning and Computer Vision,computer science data analysis engineering exc...,1144,1739
1957,data scientist college graduate,Full Time,Junior,39,Cloud Computing and Data Science,apis clustering computer science data visualiz...,1145,1740


In [148]:
result_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1959 entries, 0 to 1958
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       1959 non-null   object
 1   Type        1959 non-null   object
 2   level       1959 non-null   object
 3   Salary      1959 non-null   int64 
 4   classe      1959 non-null   object
 5   Requirment  1959 non-null   object
 6   jobs_id     1959 non-null   int64 
 7   req_id      1959 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 122.6+ KB


In [149]:
# Create the table if it doesn't exist
cursor.execute('''
CREATE TABLE Description (
    Type VARCHAR(50),
    Salary INT,
    job_id INT,
    classe VARCHAR(100),
    level VARCHAR(50),
    req_id INT,
    FOREIGN KEY (job_id) REFERENCES Jobs(job_id),
    FOREIGN KEY (req_id) REFERENCES Requirement(req_id)
)
''')
conn.commit()

# Insert data into the table
for row in result_df2.itertuples(index=False):
    cursor.execute("INSERT INTO Description (Type, Salary, job_id, classe, level, req_id ) VALUES (?, ?, ?, ?, ?, ?)",
                   row.Type, row.Salary, row.jobs_id, row.classe, row.level, row.req_id )
    conn.commit()